In [1]:
import hashlib
import json
from datetime import datetime, timedelta
import logging

In [2]:
# Configure logging to capture hotel management events
logging.basicConfig(filename='hotel_management.log', level=logging.INFO, format='%(asctime)s - %(message)s')

In [3]:
class Room:
    def __init__(self, room_number, room_type, price, capacity=None, amenities=None, status="Available"):
        """
        Initializes a Room instance with the provided details.

        :param room_number: Unique identifier for the room.
        :param room_type: Type of the room (e.g., single, double).
        :param price: Cost per night for the room.
        :param capacity: Maximum number of guests the room can accommodate (optional).
        :param amenities: List of amenities available in the room (optional).
        :param status: Availability status of the room (default is "Available").
        """
        self.room_number = room_number
        self.room_type = room_type
        self.price = price
        self.capacity = capacity
        self.amenities = amenities or []
        self.status = status
        self.bookings = []  # List to store bookings for the room

    def __str__(self):
        """
        Returns a string representation of the Room instance, detailing its attributes.
        """
        return f"Room {self.room_number} ({self.room_type}) - ${self.price}/night - {self.status}"

    def is_available_for_dates(self, start_date, end_date):
        """
        Checks if the room is available for the specified date range.

        :param start_date: Start date of the booking.
        :param end_date: End date of the booking.
        :return: True if the room is available, False otherwise.
        """
        for booking in self.bookings:
            if (start_date <= booking['end_date'] and end_date >= booking['start_date']):
                return False
        return True

In [4]:
class User:
    def __init__(self, username, password, role='staff'):
        """
        Initializes a User instance with the provided details.

        :param username: Unique identifier for the user.
        :param password: Password for the user.
        :param role: Role of the user (default is 'staff').
        """
        self.username = username
        self.password = self.hash_password(password)  # Hash the password for security
        self.role = role

    def hash_password(self, password):
        """
        Hashes the provided password using SHA-256.

        :param password: The password to hash.
        :return: Hashed password.
        """
        return hashlib.sha256(password.encode()).hexdigest()

    def verify_password(self, password):
        """
        Verifies if the provided password matches the stored hashed password.

        :param password: The password to verify.
        :return: True if the password matches, False otherwise.
        """
        return self.password == self.hash_password(password)


In [5]:
class Hotel:
    def __init__(self):
        """
        Initializes the Hotel instance, setting up rooms, bookings, users, and booking history.
        """
        self.rooms = []  # List to store all rooms
        self.bookings = {}  # Dictionary to track current bookings
        self.users = {}  # Dictionary to store registered users
        self.booking_history = []  # List to store all past bookings

    def load_rooms(self, filename):
        """
        Loads room data from a JSON file and adds the rooms to the hotel.

        :param filename: Name of the file to load room data from.
        """
        try:
            with open(filename, 'r') as file:
                rooms_data = json.load(file)
                for room in rooms_data:
                    self.add_room(Room(
                        room['room_number'],
                        room['room_type'],
                        room['price'],
                        room.get('capacity'),
                        room.get('amenities', []),
                        room.get('status', 'Available')
                    ))
        except FileNotFoundError:
            print(f"File {filename} not found.")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

    def add_room(self, room):
        """
        Adds a Room object to the hotel's list of rooms.

        :param room: The Room object to add.
        """
        self.rooms.append(room)

    def view_rooms(self):
        """
        Prints details of all rooms in the hotel.
        """
        for room in self.rooms:
            print(room)

    def book_room(self, room_number, guest_name, stay_duration, start_date):
        """
        Books a room for a guest if it is available for the specified dates.

        :param room_number: The number of the room to book.
        :param guest_name: Name of the guest.
        :param stay_duration: Duration of the stay in days.
        :param start_date: Start date of the stay.
        """
        room = next((r for r in self.rooms if str(r.room_number) == str(room_number)), None)
        if room:
            end_date = start_date + timedelta(days=stay_duration)
            if room.is_available_for_dates(start_date, end_date):
                room.bookings.append({'guest_name': guest_name, 'start_date': start_date, 'end_date': end_date})
                self.bookings[room_number] = {
                    'guest_name': guest_name,
                    'stay_duration': stay_duration,
                    'start_date': start_date,
                    'end_date': end_date,
                    'room_number': room_number
                }
                self.booking_history.append(self.bookings[room_number])
                total_cost = room.price * stay_duration
                print(f"Room {room_number} successfully booked for {guest_name}.")
                print(f"Total cost for {stay_duration} nights: ${total_cost:.2f}")
            else:
                print(f"Room {room_number} is not available for the selected dates.")
        else:
            print(f"Room {room_number} not found.")

    def cancel_booking(self, room_number, guest_name):
        """
        Cancels an existing booking for a guest in the specified room.

        :param room_number: The number of the room to cancel booking for.
        :param guest_name: Name of the guest whose booking is to be canceled.
        :return: True if cancellation was successful, False otherwise.
        """
        room = next((r for r in self.rooms if r.room_number == room_number), None)
        if room:
            for booking in room.bookings:
                if booking['guest_name'] == guest_name:
                    room.bookings.remove(booking)
                    if room_number in self.bookings:
                        del self.bookings[room_number]
                    print(f"Booking for {guest_name} in Room {room_number} has been canceled.")
                    return True
            print(f"No booking found for {guest_name} in Room {room_number}.")
        else:
            print(f"Room {room_number} not found.")
        return False

    def check_out(self, room_number):
        """
        Checks out a guest from a room, updating the room's status.

        :param room_number: The number of the room to check out from.
        """
        if room_number in self.bookings:
            room = next((r for r in self.rooms if r.room_number == room_number), None)
            if room:
                room.status = "Needs Cleaning"  # Update the room status
                del self.bookings[room_number]  # Remove booking from current bookings
                print(f"Checked out of room {room_number}.")
            else:
                print("Room not found.")
        else:
            print(f"No booking found for room {room_number}.")

    def view_bookings(self):
        """
        Displays current bookings in the hotel.
        """
        if not self.bookings:
            print("No current bookings.")
            return
        for room_number, details in self.bookings.items():
            print(f"Room {room_number} booked by {details['guest_name']} from {details['start_date']} to {details['end_date']}.")

    def view_booking_history(self):
        """
        Shows the history of all bookings made in the hotel.
        """
        if not self.booking_history:
            print("No booking history available.")
        else:
            for booking in self.booking_history:
                print(f"Room {booking['room_number']} booked by {booking['guest_name']} for {booking['stay_duration']} days.")

    def apply_discount(self, room_number, discount_percent):
        """
        Applies a discount to the specified room's price.

        :param room_number: The number of the room to apply the discount to.
        :param discount_percent: Percentage discount to apply.
        """
        room = next((r for r in self.rooms if r.room_number == room_number), None)
        if room:
            discounted_price = room.price * (1 - discount_percent / 100)
            print(f"New discounted price for room {room_number}: ${discounted_price:.2f}")
        else:
            print(f"Room {room_number} not found.")

    def load_users(self, filename):
        """
        Loads user data from a JSON file and registers users.

        :param filename: Name of the file to load user data from.
        """
        try:
            with open(filename, 'r') as file:
                users_data = json.load(file)
                for user in users_data:
                    self.users[user['username']] = User(user['username'], user['password'], user['role'])
        except FileNotFoundError:
            print(f"File {filename} not found.")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

    def save_users(self, filename):
        """
        Saves user data to a JSON file.

        :param filename: Name of the file to save user data to.
        """
        with open(filename, 'w') as file:
            json.dump([{'username': username, 'password': user.password, 'role': user.role} for username, user in self.users.items()], file)

    def register_user(self, username, password, role):
        """
        Registers a new user and saves them to a file.

        :param username: Unique identifier for the new user.
        :param password: Password for the new user.
        :param role: Role of the new user (either 'staff' or 'admin').
        """
        if username in self.users:
            print(f"Username {username} already exists.")
        else:
            self.users[username] = User(username, password, role)
            self.save_users('users.json')  # Save users to JSON after registration
            print(f"User {username} registered successfully.")

    def verify_user(self, username, password):
        """
        Verifies user credentials and returns their role if successful.

        :param username: Username of the user to verify.
        :param password: Password of the user to verify.
        :return: Role of the user if verified, None otherwise.
        """
        user = self.users.get(username)
        if user and user.verify_password(password):
            return user.role
        return None

In [6]:
# Main program loop
def main():
    """
    The main loop of the hotel management system that handles user interaction.
    """
    
    print("Welcome to Eileen Hotel Cairo Management System")
    print()
    hotel = Hotel()
    hotel.load_rooms("rooms.json")  # Load rooms from JSON file
    hotel.load_users("users.json")  # Load users from JSON file

    while True:
        print()
        print("Welcome, Choice from Menu")
        print("1. Register")
        print("2. Login")
        print("3. Quit")

        choice = input("Choose an option: ")

        if choice == '1':
            username = input("Enter username: ")
            print(f"You entered username: {username}")
            password = input("Enter password: ")
            print("Do you want to see the password? (yes/no)")
            Choice_1=input("please enter 'yes' or 'No' : ")
            if Choice_1.lower() == 'yes':
                print(f"Password entered: {password}")
            elif Choice_1.lower() == 'no':
                print("I will display password hashed to secure")
                print(f"Password entered: {'*' * len(password)}")
            else :
                print("Invalid choice")
            role = input("Enter role (admin/staff): ").lower()
            hotel.register_user(username, password, role)
            print()

        elif choice == '2':
            username = input("Enter username: ")
            print(f"You entered username: {username}")
            password = input("Enter password: ")
            print("Do you want to see the password? (yes/no)")
            Choice_1=input("please enter 'yes' or 'No' : ")
            if Choice_1.lower() == 'yes':
                print(f"Password entered: {password}")
            elif Choice_1.lower() == 'no':
                print("I will display password hashed to secure")
                print(f"Password entered: {'*' * len(password)}")
            else :
                print("Invalid choice")
            role = hotel.verify_user(username, password)
            if not role:
                print("Login failed.")
                continue
            print(f"Login successful! Role: {role}")
            print(f"Welcome {username} to Eileen Hotel Cairo Management System")
            print()

            if role == 'admin':
                print("Admin Menu")
                while True:
                    print("1. View All Rooms")
                    print("2. Book Room")
                    print("3. Cancel Booking")
                    print("4. Logout")

                    admin_choice = input("Choose an option: ")

                    if admin_choice == '1':
                        hotel.view_rooms()
                        print()
                    elif admin_choice == '2':
                        room_number = input("Enter room number to book: ")
                        guest_name = input("Enter guest name: ")
                        stay_duration = int(input("Enter stay duration (in days): "))
                        start_date = datetime.now()
                        hotel.book_room(room_number, guest_name, stay_duration, start_date)
                        print()
                    elif admin_choice == '3':
                          room_number = input("Enter room number to cancel booking: ")
                          guest_name = input("Enter guest name to cancel booking: ") # Get guest_name from user
                          hotel.cancel_booking(room_number, guest_name) # Pass both arguments
                          print()
                    elif admin_choice == '4':
                        print(" You Logging out from Admin Menu,I hope I was able to help you")
                        break
                    else:
                        print("Invalid choice.")

            elif role == 'staff':
                print("Staff Menu")
                while True:
                    print("1. View All Rooms")
                    print("2. Book Room")
                    print("3. Cancel Booking")
                    print("4. View Bookings")
                    print("5. Apply Discount")
                    print("6. Add/Remove Rooms")  #  only staff option
                    print("7. Logout")

                    staff_choice = input("Choose an option: ")

                    if staff_choice == '1':
                        hotel.view_rooms()
                        print()

                    elif staff_choice == '2':
                        room_number = input("Enter room number to book: ")
                        guest_name = input("Enter guest name: ")
                        stay_duration = int(input("Enter stay duration (in days): "))
                        start_date = datetime.now()
                        hotel.book_room(room_number, guest_name, stay_duration, start_date)
                        print()

                    elif staff_choice == '3':
                        room_number = input("Enter room number to cancel booking: ")
                        hotel.cancel_booking(room_number)
                        print()

                    elif staff_choice == '4':
                        hotel.view_bookings()
                        print()

                    elif staff_choice == '5':
                        room_number = input("Enter room number to apply discount: ")
                        discount_percent = float(input("Enter discount percentage: "))
                        hotel.apply_discount(room_number, discount_percent)
                        print()

                    elif staff_choice == '6':
                        action = input("Would you like to add or remove a room? (add/remove): ").lower()
                        if action == 'add':
                            room_number = input("Enter new room number: ")
                            room_type = input("Enter room type: ")
                            price = float(input("Enter room price: "))
                            hotel.add_room(room_number, room_type, price)  # Implement this method
                        elif action == 'remove':
                            room_number = input("Enter room number to remove: ")
                            hotel.remove_room(room_number)  # Implement this method
                        else:
                            print("Invalid option.")
                        print()

                    elif staff_choice == '7':
                        print(" You Logging out from Admin Menu,I hope I was able to help you")
                        print()
                        break
                    else:
                        print("Invalid choice.")

            else:
                print("Login failed.")

        elif choice == '3':
            print("We hope you will become our regular guest. Thank you for your loyalty.")
            print("Thank you for using Eileen Hotel Cairo Management System")
            print("Goodbye!")
            print()
            break
        else:
            print("Invalid choice.")


In [ ]:
if __name__ == "__main__":
    main()

Welcome to the Hotel Management System

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
you are enter username : 
Invalid choice
Login failed.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1. Register
2. Login
3. Quit
Invalid choice.

1